In [1]:
from pathlib import Path
from easydict import EasyDict as edict
from pathlib import Path
import torch
from torch.nn import CrossEntropyLoss
from torchvision import transforms as trans
import os
import mxnet as mx
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import Dataset, ConcatDataset, DataLoader
from torchvision import transforms as trans
from torchvision.datasets import ImageFolder
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np
import cv2
import bcolz
import pickle
import torch
import mxnet as mx
from tqdm import tqdm
import pandas as pd



# 1. train.rec, train.idx -> img

In [4]:
### config.py
def get_config(training = True):
    conf = edict()
    conf.data_path = Path('../data')
    conf.work_path = Path('./work_space/')
    conf.model_path = conf.work_path/'models'
    conf.log_path = conf.work_path/'log'
    conf.save_path = conf.work_path/'save'
    conf.input_size = [112,112]
    conf.embedding_size = 512
    conf.use_mobilfacenet = False
    conf.net_depth = 50
    conf.drop_ratio = 0.6
    conf.net_mode = 'ir_se' # or 'ir'
    conf.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    conf.test_transform = trans.Compose([
                    trans.ToTensor(),
                    trans.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                ])
    conf.data_mode = 'emore'
    conf.vgg_folder = conf.data_path/'faces_vgg_112x112'
    conf.ms1m_folder = conf.data_path/'faces_ms1m_112x112'
    conf.emore_folder = conf.data_path/'faces_emore'
    conf.batch_size = 100 # irse net depth 50 
#   conf.batch_size = 200 # mobilefacenet
#--------------------Training Config ------------------------    
    if training:        
        conf.log_path = conf.work_path/'log'
        conf.save_path = conf.work_path/'save'
    #     conf.weight_decay = 5e-4
        conf.lr = 1e-3
        conf.milestones = [12,15,18]
        conf.momentum = 0.9
        conf.pin_memory = True
#         conf.num_workers = 4 # when batchsize is 200
        conf.num_workers = 3
        conf.ce_loss = CrossEntropyLoss()    
#--------------------Inference Config ------------------------
    else:
        conf.facebank_path = conf.data_path/'facebank'
        conf.threshold = 1.5
        conf.face_limit = 10 
        #when inference, at maximum detect 10 faces in one image, my laptop is slow
        conf.min_face_size = 30 
        # the larger this value, the faster deduction, comes with tradeoff in small faces
    return conf

### data_pipe.py

In [42]:
def load_bin(path, rootdir, transform, image_size=[112,112]):
    if not rootdir.exists():
        rootdir.mkdir()
    bins, issame_list = pickle.load(open(path, 'rb'), encoding='bytes')
    data = bcolz.fill([len(bins), 3, image_size[0], image_size[1]], dtype=np.float32, rootdir=rootdir, mode='w')
    for i in range(len(bins)):
        _bin = bins[i]
        img = mx.image.imdecode(_bin).asnumpy()
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        img = Image.fromarray(img.astype(np.uint8))
        data[i, ...] = transform(img)
        i += 1
        if i % 1000 == 0:
            print('loading bin', i)
    print(data.shape)
    np.save(str(rootdir)+'_list', np.array(issame_list))
    return data, issame_list

def load_mx_rec(rec_path):
    save_path = rec_path/'imgs'
    print(save_path)
#     save_path = save_path.replace('//','/')
    if not save_path.exists():
        save_path.mkdir()
    imgrec = mx.recordio.MXIndexedRecordIO(str(rec_path/'train.idx'), str(rec_path/'train.rec'), 'r')
    img_info = imgrec.read_idx(0)
    header,_ = mx.recordio.unpack(img_info)
    max_idx = int(header.label[0])
    for idx in tqdm(range(1,max_idx)):
        img_info = imgrec.read_idx(idx)
        header, img = mx.recordio.unpack_img(img_info)
        label = int(header.label)
        
        # 원본 소스: 이미지 객체로 변환후 저장 -> bgr 채널로 저장
#         img = Image.fromarray(img)  # numpy 배열을 Image객체로 바꿀 때
#         label_path = save_path/str(label)
#         if not label_path.exists():
#             label_path.mkdir()
#         img.save(label_path/'{}.jpg'.format(idx), quality=95)

        # openCV로 numpy로 저장 -> rgb 채널순으로 저장
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label_path = save_path/str(label)
        if not label_path.exists():
            label_path.mkdir()
        cv2.imwrite(f'{label_path}/{idx}.jpg',img, [cv2.IMWRITE_JPEG_QUALITY, 95])



### prepare_data.py

In [6]:
args = edict()
args.rec_path = 'faces_webface_112x112/'
conf = get_config()
rec_path = conf.data_path/args.rec_path
print(rec_path, os.path.isdir(rec_path))

..\data\faces_webface_112x112 True


In [44]:
load_mx_rec(rec_path)

..\data\faces_webface_112x112\imgs


100%|█████████████████████████████████████████████████████████████████████████| 490623/490623 [58:29<00:00, 139.79it/s]


# 2. img -> train2.lst

In [34]:
rec_path

WindowsPath('../data/faces_webface_112x112')

In [35]:
input_dir =os.path.join(rec_path,'imgs')
print(input_dir, os.path.isdir(input_dir))


..\data\faces_webface_112x112\imgs True


In [36]:
ret = []
label = 0
person_names = []
for person_name in os.listdir(input_dir):
    person_names.append(person_name)
person_names = sorted(person_names)

for person_name in person_names:
    # persondir_path
    _subdir = os.path.join(input_dir, person_name)
    print(_subdir)
    if not os.path.isdir(_subdir):
        continue
    
    _ret = []
    for img in os.listdir(_subdir):
        # imgfile_path
        fimage = edict()
        fimage.id = os.path.join(_subdir, img)
        fimage.classname = str(label)
        fimage.image_path = os.path.join(_subdir, img)
        fimage.bbox = None
        fimage.landmark = None
        _ret.append(fimage)
    ret += _ret
    label+=1
        
            

..\data\faces_webface_112x112\imgs\0
..\data\faces_webface_112x112\imgs\1
..\data\faces_webface_112x112\imgs\10
..\data\faces_webface_112x112\imgs\100
..\data\faces_webface_112x112\imgs\1000
..\data\faces_webface_112x112\imgs\10000
..\data\faces_webface_112x112\imgs\10001
..\data\faces_webface_112x112\imgs\10002
..\data\faces_webface_112x112\imgs\10003
..\data\faces_webface_112x112\imgs\10004
..\data\faces_webface_112x112\imgs\10005
..\data\faces_webface_112x112\imgs\10006
..\data\faces_webface_112x112\imgs\10007
..\data\faces_webface_112x112\imgs\10008
..\data\faces_webface_112x112\imgs\10009
..\data\faces_webface_112x112\imgs\1001
..\data\faces_webface_112x112\imgs\10010
..\data\faces_webface_112x112\imgs\10011
..\data\faces_webface_112x112\imgs\10012
..\data\faces_webface_112x112\imgs\10013
..\data\faces_webface_112x112\imgs\10014
..\data\faces_webface_112x112\imgs\10015
..\data\faces_webface_112x112\imgs\10016
..\data\faces_webface_112x112\imgs\10017
..\data\faces_webface_112x112\i

..\data\faces_webface_112x112\imgs\10224
..\data\faces_webface_112x112\imgs\10225
..\data\faces_webface_112x112\imgs\10226
..\data\faces_webface_112x112\imgs\10227
..\data\faces_webface_112x112\imgs\10228
..\data\faces_webface_112x112\imgs\10229
..\data\faces_webface_112x112\imgs\1023
..\data\faces_webface_112x112\imgs\10230
..\data\faces_webface_112x112\imgs\10231
..\data\faces_webface_112x112\imgs\10232
..\data\faces_webface_112x112\imgs\10233
..\data\faces_webface_112x112\imgs\10234
..\data\faces_webface_112x112\imgs\10235
..\data\faces_webface_112x112\imgs\10236
..\data\faces_webface_112x112\imgs\10237
..\data\faces_webface_112x112\imgs\10238
..\data\faces_webface_112x112\imgs\10239
..\data\faces_webface_112x112\imgs\1024
..\data\faces_webface_112x112\imgs\10240
..\data\faces_webface_112x112\imgs\10241
..\data\faces_webface_112x112\imgs\10242
..\data\faces_webface_112x112\imgs\10243
..\data\faces_webface_112x112\imgs\10244
..\data\faces_webface_112x112\imgs\10245
..\data\faces_webf

..\data\faces_webface_112x112\imgs\10443
..\data\faces_webface_112x112\imgs\10444
..\data\faces_webface_112x112\imgs\10445
..\data\faces_webface_112x112\imgs\10446
..\data\faces_webface_112x112\imgs\10447
..\data\faces_webface_112x112\imgs\10448
..\data\faces_webface_112x112\imgs\10449
..\data\faces_webface_112x112\imgs\1045
..\data\faces_webface_112x112\imgs\10450
..\data\faces_webface_112x112\imgs\10451
..\data\faces_webface_112x112\imgs\10452
..\data\faces_webface_112x112\imgs\10453
..\data\faces_webface_112x112\imgs\10454
..\data\faces_webface_112x112\imgs\10455
..\data\faces_webface_112x112\imgs\10456
..\data\faces_webface_112x112\imgs\10457
..\data\faces_webface_112x112\imgs\10458
..\data\faces_webface_112x112\imgs\10459
..\data\faces_webface_112x112\imgs\1046
..\data\faces_webface_112x112\imgs\10460
..\data\faces_webface_112x112\imgs\10461
..\data\faces_webface_112x112\imgs\10462
..\data\faces_webface_112x112\imgs\10463
..\data\faces_webface_112x112\imgs\10464
..\data\faces_webf

..\data\faces_webface_112x112\imgs\1135
..\data\faces_webface_112x112\imgs\1136
..\data\faces_webface_112x112\imgs\1137
..\data\faces_webface_112x112\imgs\1138
..\data\faces_webface_112x112\imgs\1139
..\data\faces_webface_112x112\imgs\114
..\data\faces_webface_112x112\imgs\1140
..\data\faces_webface_112x112\imgs\1141
..\data\faces_webface_112x112\imgs\1142
..\data\faces_webface_112x112\imgs\1143
..\data\faces_webface_112x112\imgs\1144
..\data\faces_webface_112x112\imgs\1145
..\data\faces_webface_112x112\imgs\1146
..\data\faces_webface_112x112\imgs\1147
..\data\faces_webface_112x112\imgs\1148
..\data\faces_webface_112x112\imgs\1149
..\data\faces_webface_112x112\imgs\115
..\data\faces_webface_112x112\imgs\1150
..\data\faces_webface_112x112\imgs\1151
..\data\faces_webface_112x112\imgs\1152
..\data\faces_webface_112x112\imgs\1153
..\data\faces_webface_112x112\imgs\1154
..\data\faces_webface_112x112\imgs\1155
..\data\faces_webface_112x112\imgs\1156
..\data\faces_webface_112x112\imgs\1157
..

..\data\faces_webface_112x112\imgs\1381
..\data\faces_webface_112x112\imgs\1382
..\data\faces_webface_112x112\imgs\1383
..\data\faces_webface_112x112\imgs\1384
..\data\faces_webface_112x112\imgs\1385
..\data\faces_webface_112x112\imgs\1386
..\data\faces_webface_112x112\imgs\1387
..\data\faces_webface_112x112\imgs\1388
..\data\faces_webface_112x112\imgs\1389
..\data\faces_webface_112x112\imgs\139
..\data\faces_webface_112x112\imgs\1390
..\data\faces_webface_112x112\imgs\1391
..\data\faces_webface_112x112\imgs\1392
..\data\faces_webface_112x112\imgs\1393
..\data\faces_webface_112x112\imgs\1394
..\data\faces_webface_112x112\imgs\1395
..\data\faces_webface_112x112\imgs\1396
..\data\faces_webface_112x112\imgs\1397
..\data\faces_webface_112x112\imgs\1398
..\data\faces_webface_112x112\imgs\1399
..\data\faces_webface_112x112\imgs\14
..\data\faces_webface_112x112\imgs\140
..\data\faces_webface_112x112\imgs\1400
..\data\faces_webface_112x112\imgs\1401
..\data\faces_webface_112x112\imgs\1402
..\d

...

..\data\faces_webface_112x112\imgs\9068
..\data\faces_webface_112x112\imgs\9069
..\data\faces_webface_112x112\imgs\907
..\data\faces_webface_112x112\imgs\9070
..\data\faces_webface_112x112\imgs\9071
..\data\faces_webface_112x112\imgs\9072
..\data\faces_webface_112x112\imgs\9073
..\data\faces_webface_112x112\imgs\9074
..\data\faces_webface_112x112\imgs\9075
..\data\faces_webface_112x112\imgs\9076
..\data\faces_webface_112x112\imgs\9077
..\data\faces_webface_112x112\imgs\9078
..\data\faces_webface_112x112\imgs\9079
..\data\faces_webface_112x112\imgs\908
..\data\faces_webface_112x112\imgs\9080
..\data\faces_webface_112x112\imgs\9081
..\data\faces_webface_112x112\imgs\9082
..\data\faces_webface_112x112\imgs\9083
..\data\faces_webface_112x112\imgs\9084
..\data\faces_webface_112x112\imgs\9085
..\data\faces_webface_112x112\imgs\9086
..\data\faces_webface_112x112\imgs\9087
..\data\faces_webface_112x112\imgs\9088
..\data\faces_webface_112x112\imgs\9089
..\data\faces_webface_112x112\imgs\909
..\

..\data\faces_webface_112x112\imgs\9299
..\data\faces_webface_112x112\imgs\93
..\data\faces_webface_112x112\imgs\930
..\data\faces_webface_112x112\imgs\9300
..\data\faces_webface_112x112\imgs\9301
..\data\faces_webface_112x112\imgs\9302
..\data\faces_webface_112x112\imgs\9303
..\data\faces_webface_112x112\imgs\9304
..\data\faces_webface_112x112\imgs\9305
..\data\faces_webface_112x112\imgs\9306
..\data\faces_webface_112x112\imgs\9307
..\data\faces_webface_112x112\imgs\9308
..\data\faces_webface_112x112\imgs\9309
..\data\faces_webface_112x112\imgs\931
..\data\faces_webface_112x112\imgs\9310
..\data\faces_webface_112x112\imgs\9311
..\data\faces_webface_112x112\imgs\9312
..\data\faces_webface_112x112\imgs\9313
..\data\faces_webface_112x112\imgs\9314
..\data\faces_webface_112x112\imgs\9315
..\data\faces_webface_112x112\imgs\9316
..\data\faces_webface_112x112\imgs\9317
..\data\faces_webface_112x112\imgs\9318
..\data\faces_webface_112x112\imgs\9319
..\data\faces_webface_112x112\imgs\932
..\da

..\data\faces_webface_112x112\imgs\9546
..\data\faces_webface_112x112\imgs\9547
..\data\faces_webface_112x112\imgs\9548
..\data\faces_webface_112x112\imgs\9549
..\data\faces_webface_112x112\imgs\955
..\data\faces_webface_112x112\imgs\9550
..\data\faces_webface_112x112\imgs\9551
..\data\faces_webface_112x112\imgs\9552
..\data\faces_webface_112x112\imgs\9553
..\data\faces_webface_112x112\imgs\9554
..\data\faces_webface_112x112\imgs\9555
..\data\faces_webface_112x112\imgs\9556
..\data\faces_webface_112x112\imgs\9557
..\data\faces_webface_112x112\imgs\9558
..\data\faces_webface_112x112\imgs\9559
..\data\faces_webface_112x112\imgs\956
..\data\faces_webface_112x112\imgs\9560
..\data\faces_webface_112x112\imgs\9561
..\data\faces_webface_112x112\imgs\9562
..\data\faces_webface_112x112\imgs\9563
..\data\faces_webface_112x112\imgs\9564
..\data\faces_webface_112x112\imgs\9565
..\data\faces_webface_112x112\imgs\9566
..\data\faces_webface_112x112\imgs\9567
..\data\faces_webface_112x112\imgs\9568
..

..\data\faces_webface_112x112\imgs\9775
..\data\faces_webface_112x112\imgs\9776
..\data\faces_webface_112x112\imgs\9777
..\data\faces_webface_112x112\imgs\9778
..\data\faces_webface_112x112\imgs\9779
..\data\faces_webface_112x112\imgs\978
..\data\faces_webface_112x112\imgs\9780
..\data\faces_webface_112x112\imgs\9781
..\data\faces_webface_112x112\imgs\9782
..\data\faces_webface_112x112\imgs\9783
..\data\faces_webface_112x112\imgs\9784
..\data\faces_webface_112x112\imgs\9785
..\data\faces_webface_112x112\imgs\9786
..\data\faces_webface_112x112\imgs\9787
..\data\faces_webface_112x112\imgs\9788
..\data\faces_webface_112x112\imgs\9789
..\data\faces_webface_112x112\imgs\979
..\data\faces_webface_112x112\imgs\9790
..\data\faces_webface_112x112\imgs\9791
..\data\faces_webface_112x112\imgs\9792
..\data\faces_webface_112x112\imgs\9793
..\data\faces_webface_112x112\imgs\9794
..\data\faces_webface_112x112\imgs\9795
..\data\faces_webface_112x112\imgs\9796
..\data\faces_webface_112x112\imgs\9797
..

..\data\faces_webface_112x112\imgs\9973
..\data\faces_webface_112x112\imgs\9974
..\data\faces_webface_112x112\imgs\9975
..\data\faces_webface_112x112\imgs\9976
..\data\faces_webface_112x112\imgs\9977
..\data\faces_webface_112x112\imgs\9978
..\data\faces_webface_112x112\imgs\9979
..\data\faces_webface_112x112\imgs\998
..\data\faces_webface_112x112\imgs\9980
..\data\faces_webface_112x112\imgs\9981
..\data\faces_webface_112x112\imgs\9982
..\data\faces_webface_112x112\imgs\9983
..\data\faces_webface_112x112\imgs\9984
..\data\faces_webface_112x112\imgs\9985
..\data\faces_webface_112x112\imgs\9986
..\data\faces_webface_112x112\imgs\9987
..\data\faces_webface_112x112\imgs\9988
..\data\faces_webface_112x112\imgs\9989
..\data\faces_webface_112x112\imgs\999
..\data\faces_webface_112x112\imgs\9990
..\data\faces_webface_112x112\imgs\9991
..\data\faces_webface_112x112\imgs\9992
..\data\faces_webface_112x112\imgs\9993
..\data\faces_webface_112x112\imgs\9994
..\data\faces_webface_112x112\imgs\9995
..

In [37]:
lst_save = os.path.join(rec_path, 'train2.lst')
with open(lst_save, 'a', encoding = 'utf-8') as f:
    for item in ret:
        item.image_path = item.image_path.replace('\\', '/')
        f.write("%d\t%s\t%d\n" % (1, item.image_path, int(item.classname)))
        # 정렬여부, 절대경로, label

# 3. train.lst vs train2.lst
- train.lst: download 받은 파일
- train2.lst : 내가 생성한 파일

In [7]:
train_path = os.path.join(rec_path, 'train.lst')
train2_path = os.path.join(rec_path, 'train2.lst')

train_data = pd.read_csv(train_path,sep = '\t',
                        names =['aligned', 'img_path', 'label', 'bbox_x1', 'bbox_y1','bbox_x2', 'bbox_y2', 'p1', 'p2','p3', 'p4', 'p5',
                                'p6', 'p7', 'p8', 'p9', 'p10'])
train2_data = pd.read_csv(train2_path,sep = '\t', names = ['align', 'img_path', 'label'])

In [8]:
train_data.head()

,aligned,img_path,label,bbox_x1,bbox_y1,bbox_x2,bbox_y2,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10
0,0,/raid5data/dplearn/CASIA-WebFace/0000045/001.jpg,0,57.0,59.0,187.0,212.0,93.3661,151.566,127.419,107.7320,158.046,115.133,105.1330,139.307,179.872,170.261
1,0,/raid5data/dplearn/CASIA-WebFace/0000045/013.jpg,0,55.0,50.0,168.0,199.0,97.5698,152.193,132.470,104.2200,147.564,102.114,99.1113,140.006,169.167,166.637
2,0,/raid5data/dplearn/CASIA-WebFace/0000045/012.jpg,0,57.0,45.0,187.0,218.0,92.6203,150.450,123.761,106.6710,149.209,109.181,103.0160,141.798,179.750,174.899
3,0,/raid5data/dplearn/CASIA-WebFace/0000045/009.jpg,0,66.0,43.0,193.0,211.0,95.5066,152.310,124.884,105.5960,155.331,112.456,106.3130,145.195,172.543,167.151
4,0,/raid5data/dplearn/CASIA-WebFace/0000045/010.jpg,0,75.0,43.0,187.0,198.0,102.7500,152.669,116.163,92.0129,131.055,97.403,109.9640,139.563,161.541,172.137


In [9]:
train_data.tail()

,aligned,img_path,label,bbox_x1,bbox_y1,bbox_x2,bbox_y2,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10
494144,0,/raid5data/dplearn/CASIA-WebFace/6573530/029.jpg,10571,52.0,32.0,172.0,211.0,93.2538,148.738,137.4350,102.7340,154.435,111.628,100.4290,147.422,170.765,158.917
494145,0,/raid5data/dplearn/CASIA-WebFace/6573530/044.jpg,10571,54.0,39.0,185.0,204.0,93.4072,149.741,134.4840,113.0520,157.873,114.141,98.8091,138.303,171.445,158.705
494146,0,/raid5data/dplearn/CASIA-WebFace/6573530/045.jpg,10571,81.0,48.0,186.0,190.0,97.7086,145.643,111.7820,102.6780,139.522,105.311,106.0800,130.119,158.789,159.863
494147,0,/raid5data/dplearn/CASIA-WebFace/6573530/023.jpg,10571,57.0,41.0,188.0,203.0,95.0552,151.573,141.8370,118.8150,172.882,120.013,96.7601,140.140,175.517,153.446
494148,0,/raid5data/dplearn/CASIA-WebFace/6573530/054.jpg,10571,79.0,43.0,186.0,200.0,99.2262,130.752,88.6318,88.5708,111.059,105.544,110.2790,136.087,170.498,174.095


In [10]:
train2_data.head()

,align,img_path,label
0,1,../data/faces_webface_112x112/imgs/0/1.jpg,0
1,1,../data/faces_webface_112x112/imgs/0/10.jpg,0
2,1,../data/faces_webface_112x112/imgs/0/11.jpg,0
3,1,../data/faces_webface_112x112/imgs/0/12.jpg,0
4,1,../data/faces_webface_112x112/imgs/0/13.jpg,0


In [11]:
train2_data.tail()

,align,img_path,label
490618,1,../data/faces_webface_112x112/imgs/9999/474744...,10571
490619,1,../data/faces_webface_112x112/imgs/9999/474745...,10571
490620,1,../data/faces_webface_112x112/imgs/9999/474746...,10571
490621,1,../data/faces_webface_112x112/imgs/9999/474747...,10571
490622,1,../data/faces_webface_112x112/imgs/9999/474748...,10571


In [12]:
len(train_data)

494149

In [13]:
len(train2_data)

490623

### imgs
- 파일 490,623, 폴더 10,572

# 4. img_align 여부 확인 => align 되어있음.

In [17]:
img_path = '../data/faces_webface_112x112/imgs/0/1.jpg'
print(os.path.isfile(img))

True


In [18]:
img = cv2.imread(img_path)

In [19]:
img.shape

(112, 112, 3)

# 5. property: download 받은 파일과 내가 만든 파일이 모두 같음

In [28]:
proper_path = '../data/faces_webface_112x112/property'
with open(proper_path, 'r') as f:
    property_data = f.read()

In [29]:
property_data

'10572,112,112\n'

# 5. train2.lst -> train2.rec, train2.idx만들기

In [20]:
import os
import sys

#curr_path = os.path.abspath(os.path.dirname(__file__))
#sys.path.append(os.path.join(curr_path, "../python"))
import mxnet as mx
from skimage import transform as trans


import random
import argparse
import cv2
import time, json
import numpy as np
import traceback
#from builtins import range
from easydict import EasyDict as edict
# sys.path.append(os.path.join(os.path.dirname(__file__), '..', 'common'))
# import face_preprocess
# import face_image

try:
    import multiprocessing
except ImportError:
    multiprocessing = None

### face_process.py

In [30]:
def parse_lst_line(line):
  vec = line.strip().split("\t")
  assert len(vec)>=3
  aligned = int(vec[0])
  image_path = vec[1]
  label = int(vec[2])
  bbox = None
  landmark = None
  #print(vec)
  if len(vec)>3:
    bbox = np.zeros( (4,), dtype=np.int32)
    for i in xrange(3,7):
      bbox[i-3] = int(vec[i])
    landmark = None
    if len(vec)>7:
      _l = []
      for i in xrange(7,17):
        _l.append(float(vec[i]))
      landmark = np.array(_l).reshape( (2,5) ).T
  #print(aligned)
  return image_path, label, bbox, landmark, aligned

In [31]:
def read_image(img_path, **kwargs):
  mode = kwargs.get('mode', 'rgb')
  layout = kwargs.get('layout', 'HWC')
  if mode=='gray':
    img = cv2.imread(img_path, cv2.CV_LOAD_IMAGE_GRAYSCALE)
  else:
    img = cv2.imread(img_path, cv2.CV_LOAD_IMAGE_COLOR)
    if mode=='rgb':
      #print('to rgb')
      img = img[...,::-1]
    if layout=='CHW':
      img = np.transpose(img, (2,0,1))
  return img

In [32]:
def preprocess(img, bbox=None, landmark=None, **kwargs):
  if isinstance(img, str):
    img = read_image(img, **kwargs)
  M = None
  image_size = []
  str_image_size = kwargs.get('image_size', '')
  if len(str_image_size)>0:
    image_size = [int(x) for x in str_image_size.split(',')]
    if len(image_size)==1:
      image_size = [image_size[0], image_size[0]]
    assert len(image_size)==2
    assert image_size[0]==112
    assert image_size[0]==112 or image_size[1]==96
  if landmark is not None:
    assert len(image_size)==2
    src = np.array([
      [30.2946, 51.6963],
      [65.5318, 51.5014],
      [48.0252, 71.7366],
      [33.5493, 92.3655],
      [62.7299, 92.2041] ], dtype=np.float32 )
    if image_size[1]==112:
      src[:,0] += 8.0
    dst = landmark.astype(np.float32)

    tform = trans.SimilarityTransform()
    tform.estimate(dst, src)
    M = tform.params[0:2,:]
    #M = cv2.estimateRigidTransform( dst.reshape(1,5,2), src.reshape(1,5,2), False)

  if M is None:
    if bbox is None: #use center crop
      det = np.zeros(4, dtype=np.int32)
      det[0] = int(img.shape[1]*0.0625)
      det[1] = int(img.shape[0]*0.0625)
      det[2] = img.shape[1] - det[0]
      det[3] = img.shape[0] - det[1]
    else:
      det = bbox
    margin = kwargs.get('margin', 44)
    bb = np.zeros(4, dtype=np.int32)
    bb[0] = np.maximum(det[0]-margin/2, 0)
    bb[1] = np.maximum(det[1]-margin/2, 0)
    bb[2] = np.minimum(det[2]+margin/2, img.shape[1])
    bb[3] = np.minimum(det[3]+margin/2, img.shape[0])
    ret = img[bb[1]:bb[3],bb[0]:bb[2],:]
    if len(image_size)>0:
      ret = cv2.resize(ret, (image_size[1], image_size[0]))
    return ret 
  else: #do align using landmark
    assert len(image_size)==2

    #src = src[0:3,:]
    #dst = dst[0:3,:]


    #print(src.shape, dst.shape)
    #print(src)
    #print(dst)
    #print(M)
    warped = cv2.warpAffine(img,M,(image_size[1],image_size[0]), borderValue = 0.0)

    #tform3 = trans.ProjectiveTransform()
    #tform3.estimate(src, dst)
    #warped = trans.warp(img, tform3, output_shape=_shape)
    return warped

### face_image.py

In [33]:
def load_property(data_dir):
  prop = edict()
  for line in open(os.path.join(data_dir, 'property')):
    vec = line.strip().split(',')
    assert len(vec)==3
    prop.num_classes = int(vec[0])
    prop.image_size = [int(vec[1]), int(vec[2])]
  return prop

### face2rec2.py

In [34]:
def read_list(path_in):
    with open(path_in) as fin:
        identities = []
        last = [-1, -1]
        _id = 1
        while True:
            line = fin.readline()
            if not line:
                break
            item = edict()
            
            # flog = 0
            item.flag = 0
            item.image_path, label, item.bbox, item.landmark, item.aligned = parse_lst_line(line)
            if not item.aligned and item.landmark is None:
              #print('ignore line', line)
              continue
            item.id = _id
            item.label = [label, item.aligned]
            yield item
            if label!=last[0]:
              if last[1]>=0:
                identities.append( (last[1], _id) )
              last[0] = label
              last[1] = _id
            _id+=1
        identities.append( (last[1], _id) )
        item = edict()
        
        # flag == 2
        item.flag = 2
        item.id = 0
        item.label = [float(_id), float(_id+len(identities))]
        yield item
        for identity in identities:
          item = edict()
          item.flag = 2
          item.id = _id
          _id+=1
          item.label = [float(identity[0]), float(identity[1])]
          yield item

In [35]:
def image_encode(args, i, item, q_out):
#     print('flag:',item.flag)
    oitem = [item.id]
    
    # 개별 이미지에 대한 정보
    if item.flag==0:
        fullpath = item.image_path
        header = mx.recordio.IRHeader(item.flag, item.label, item.id, 0)
#         print('header:',header)
        
        # align이 되어있을 때(112 * 112)
        if item.aligned:
            with open(fullpath, 'rb') as fin:
                img = fin.read()
            s = mx.recordio.pack(header, img)
#             print('s',s)
            q_out.put((i, s, oitem))
            
        # align이 되어있지 않을 때
        else:
            img = cv2.imread(fullpath, args.color)
            assert item.landmark is not None
            img = preprocess(img, bbox = item.bbox, landmark=item.landmark, image_size='%d,%d'%(args.image_h, args.image_w))
            s = mx.recordio.pack_img(header, img, quality=args.quality, img_fmt=args.encoding)
            
#             print('s', s)
            q_out.put((i, s, oitem))
            
    # dir에 대한 메타 정보
    else: 
        header = mx.recordio.IRHeader(item.flag, item.label, item.id, 0)
#         print('header:', header)
        #print('write', item.flag, item.id, item.label)
        s = mx.recordio.pack(header, b'')
#         print('s', s)
        q_out.put((i, s, oitem))

# main()

In [43]:
args = edict({
    'image_h': 112,
    'image_w': 112,
    'prefix': '../data/faces_webface_112x112/',
    'root': '../data/faces_webface_112x112/imgs',
    'encoding':'.jpg',
    'quality': 95,
})

In [44]:
working_dir = args.prefix
prop = load_property(working_dir)
image_size = prop.image_size
print('image_size', image_size)

image_size [112, 112]


In [45]:
files = [os.path.join(working_dir, fname).replace('\\','/') for fname in os.listdir(working_dir)
        if os.path.isfile(os.path.join(working_dir, fname))]
print(files)

['../data/faces_webface_112x112/property', '../data/faces_webface_112x112/train2.lst']


In [46]:
count = 0
for fname in files:
    # .lst file
    if fname.startswith(args.prefix) and fname.endswith('.lst'):
        print('Creating .rec file from', fname, 'in', working_dir)
        count += 1
        image_list = read_list(fname)

        # -- write_record -- #
        try:
            import Queue as queue
        except ImportError:
            import queue
        q_out = queue.Queue()
        fname = os.path.basename(fname)
        fname_rec = os.path.splitext(fname)[0] + '.rec'
        fname_idx = os.path.splitext(fname)[0] + '.idx'
        record = mx.recordio.MXIndexedRecordIO(os.path.join(working_dir, fname_idx),
                                               os.path.join(working_dir, fname_rec), 'w')
        cnt = 0
        pre_time = time.time()
        for i, item in enumerate(image_list):
            image_encode(args, i, item, q_out)
            if q_out.empty():
                continue
            _, s, item = q_out.get()
            #header, _ = mx.recordio.unpack(s)
            #print('write header label', header.label)
#             print(item[0])
            record.write_idx(item[0], s)
            if cnt % 1000 == 0:
                cur_time = time.time()
                print('time:', cur_time - pre_time, ' count:', cnt)
                pre_time = cur_time
            cnt += 1

Creating .rec file from ../data/faces_webface_112x112/train2.lst in ../data/faces_webface_112x112/
time: 0.001003265380859375  count: 0
time: 0.8769948482513428  count: 1000
time: 1.0450024604797363  count: 2000
time: 0.8599977493286133  count: 3000
time: 0.7819993495941162  count: 4000
time: 0.875999927520752  count: 5000
time: 0.8790013790130615  count: 6000
time: 0.9329988956451416  count: 7000
time: 0.8440017700195312  count: 8000
time: 1.130000114440918  count: 9000
time: 0.9330003261566162  count: 10000
time: 0.8620009422302246  count: 11000
time: 0.7419967651367188  count: 12000
time: 0.7530002593994141  count: 13000
time: 0.7120003700256348  count: 14000
time: 0.7620019912719727  count: 15000
time: 0.8979990482330322  count: 16000
time: 0.8579998016357422  count: 17000
time: 0.7359969615936279  count: 18000
time: 0.7180018424987793  count: 19000
time: 0.7700033187866211  count: 20000
time: 0.7169969081878662  count: 21000
time: 0.7380008697509766  count: 22000
time: 0.752005577

time: 0.9679887294769287  count: 206000
time: 1.0359992980957031  count: 207000
time: 0.9260282516479492  count: 208000
time: 0.8160064220428467  count: 209000
time: 0.8549635410308838  count: 210000
time: 0.7819995880126953  count: 211000
time: 0.8470408916473389  count: 212000
time: 1.081960916519165  count: 213000
time: 1.0050017833709717  count: 214000
time: 0.8720035552978516  count: 215000
time: 0.9009933471679688  count: 216000
time: 0.8459997177124023  count: 217000
time: 0.779000997543335  count: 218000
time: 0.7749989032745361  count: 219000
time: 1.0610005855560303  count: 220000
time: 0.9260005950927734  count: 221000
time: 0.8609967231750488  count: 222000
time: 0.8860025405883789  count: 223000
time: 0.756028413772583  count: 224000
time: 0.7539980411529541  count: 225000
time: 0.7540009021759033  count: 226000
time: 1.2229762077331543  count: 227000
time: 0.8789973258972168  count: 228000
time: 0.8269996643066406  count: 229000
time: 0.8289995193481445  count: 230000
tim

time: 1.0259976387023926  count: 412000
time: 0.8180034160614014  count: 413000
time: 0.8939979076385498  count: 414000
time: 0.9339978694915771  count: 415000
time: 0.9310011863708496  count: 416000
time: 0.879997968673706  count: 417000
time: 0.9640038013458252  count: 418000
time: 1.089998483657837  count: 419000
time: 1.1051406860351562  count: 420000
time: 0.8759989738464355  count: 421000
time: 0.9620022773742676  count: 422000
time: 1.015998125076294  count: 423000
time: 1.1110048294067383  count: 424000
time: 0.8679952621459961  count: 425000
time: 0.8440039157867432  count: 426000
time: 1.098999261856079  count: 427000
time: 1.0160245895385742  count: 428000
time: 0.9039754867553711  count: 429000
time: 1.1289987564086914  count: 430000
time: 1.0209991931915283  count: 431000
time: 0.9349994659423828  count: 432000
time: 0.9080004692077637  count: 433000
time: 0.9659996032714844  count: 434000
time: 0.9270045757293701  count: 435000
time: 0.98199462890625  count: 436000
time: 

In [47]:
import os
import sys

#curr_path = os.path.abspath(os.path.dirname(__file__))
#sys.path.append(os.path.join(curr_path, "../python"))
import mxnet as mx
from skimage import transform as trans


import random
import argparse
import cv2
import time, json
import numpy as np
import traceback
#from builtins import range
from easydict import EasyDict as edict
# sys.path.append(os.path.join(os.path.dirname(__file__), '..', 'common'))
# import face_preprocess
# import face_image

try:
    import multiprocessing
except ImportError:
    multiprocessing = None

### face_process.py

def parse_lst_line(line):
  vec = line.strip().split("\t")
  assert len(vec)>=3
  aligned = int(vec[0])
  image_path = vec[1]
  label = int(vec[2])
  bbox = None
  landmark = None
  #print(vec)
  if len(vec)>3:
    bbox = np.zeros( (4,), dtype=np.int32)
    for i in xrange(3,7):
      bbox[i-3] = int(vec[i])
    landmark = None
    if len(vec)>7:
      _l = []
      for i in xrange(7,17):
        _l.append(float(vec[i]))
      landmark = np.array(_l).reshape( (2,5) ).T
  #print(aligned)
  return image_path, label, bbox, landmark, aligned

def read_image(img_path, **kwargs):
  mode = kwargs.get('mode', 'rgb')
  layout = kwargs.get('layout', 'HWC')
  if mode=='gray':
    img = cv2.imread(img_path, cv2.CV_LOAD_IMAGE_GRAYSCALE)
  else:
    img = cv2.imread(img_path, cv2.CV_LOAD_IMAGE_COLOR)
    if mode=='rgb':
      #print('to rgb')
      img = img[...,::-1]
    if layout=='CHW':
      img = np.transpose(img, (2,0,1))
  return img

def preprocess(img, bbox=None, landmark=None, **kwargs):
  if isinstance(img, str):
    img = read_image(img, **kwargs)
  M = None
  image_size = []
  str_image_size = kwargs.get('image_size', '')
  if len(str_image_size)>0:
    image_size = [int(x) for x in str_image_size.split(',')]
    if len(image_size)==1:
      image_size = [image_size[0], image_size[0]]
    assert len(image_size)==2
    assert image_size[0]==112
    assert image_size[0]==112 or image_size[1]==96
  if landmark is not None:
    assert len(image_size)==2
    src = np.array([
      [30.2946, 51.6963],
      [65.5318, 51.5014],
      [48.0252, 71.7366],
      [33.5493, 92.3655],
      [62.7299, 92.2041] ], dtype=np.float32 )
    if image_size[1]==112:
      src[:,0] += 8.0
    dst = landmark.astype(np.float32)

    tform = trans.SimilarityTransform()
    tform.estimate(dst, src)
    M = tform.params[0:2,:]
    #M = cv2.estimateRigidTransform( dst.reshape(1,5,2), src.reshape(1,5,2), False)

  if M is None:
    if bbox is None: #use center crop
      det = np.zeros(4, dtype=np.int32)
      det[0] = int(img.shape[1]*0.0625)
      det[1] = int(img.shape[0]*0.0625)
      det[2] = img.shape[1] - det[0]
      det[3] = img.shape[0] - det[1]
    else:
      det = bbox
    margin = kwargs.get('margin', 44)
    bb = np.zeros(4, dtype=np.int32)
    bb[0] = np.maximum(det[0]-margin/2, 0)
    bb[1] = np.maximum(det[1]-margin/2, 0)
    bb[2] = np.minimum(det[2]+margin/2, img.shape[1])
    bb[3] = np.minimum(det[3]+margin/2, img.shape[0])
    ret = img[bb[1]:bb[3],bb[0]:bb[2],:]
    if len(image_size)>0:
      ret = cv2.resize(ret, (image_size[1], image_size[0]))
    return ret 
  else: #do align using landmark
    assert len(image_size)==2

    #src = src[0:3,:]
    #dst = dst[0:3,:]


    #print(src.shape, dst.shape)
    #print(src)
    #print(dst)
    #print(M)
    warped = cv2.warpAffine(img,M,(image_size[1],image_size[0]), borderValue = 0.0)

    #tform3 = trans.ProjectiveTransform()
    #tform3.estimate(src, dst)
    #warped = trans.warp(img, tform3, output_shape=_shape)
    return warped

### face_image.py

def load_property(data_dir):
  prop = edict()
  for line in open(os.path.join(data_dir, 'property')):
    vec = line.strip().split(',')
    assert len(vec)==3
    prop.num_classes = int(vec[0])
    prop.image_size = [int(vec[1]), int(vec[2])]
  return prop

### face2rec2.py

def read_list(path_in):
    with open(path_in) as fin:
        identities = []
        last = [-1, -1]
        _id = 1
        while True:
            line = fin.readline()
            if not line:
                break
            item = edict()
            
            # flog = 0
            item.flag = 0
            item.image_path, label, item.bbox, item.landmark, item.aligned = parse_lst_line(line)
            if not item.aligned and item.landmark is None:
              #print('ignore line', line)
              continue
            item.id = _id
            item.label = [label, item.aligned]
            yield item
            if label!=last[0]:
              if last[1]>=0:
                identities.append( (last[1], _id) )
              last[0] = label
              last[1] = _id
            _id+=1
        identities.append( (last[1], _id) )
        item = edict()
        
        # flag == 2
        item.flag = 2
        item.id = 0
        item.label = [float(_id), float(_id+len(identities))]
        yield item
        for identity in identities:
          item = edict()
          item.flag = 2
          item.id = _id
          _id+=1
          item.label = [float(identity[0]), float(identity[1])]
          yield item

def image_encode(args, i, item, q_out):
#     print('flag:',item.flag)
    oitem = [item.id]
    
    # 개별 이미지에 대한 정보
    if item.flag==0:
        fullpath = item.image_path
        header = mx.recordio.IRHeader(item.flag, item.label, item.id, 0)
#         print('header:',header)
        
        # align이 되어있을 때(112 * 112)
        if item.aligned:
            with open(fullpath, 'rb') as fin:
                img = fin.read()
            s = mx.recordio.pack(header, img)
#             print('s',s)
            q_out.put((i, s, oitem))
            
        # align이 되어있지 않을 때
        else:
            img = cv2.imread(fullpath, args.color)
            assert item.landmark is not None
            img = preprocess(img, bbox = item.bbox, landmark=item.landmark, image_size='%d,%d'%(args.image_h, args.image_w))
            s = mx.recordio.pack_img(header, img, quality=args.quality, img_fmt=args.encoding)
            
#             print('s', s)
            q_out.put((i, s, oitem))
            
    # dir에 대한 메타 정보
    else: 
        header = mx.recordio.IRHeader(item.flag, item.label, item.id, 0)
#         print('header:', header)
        #print('write', item.flag, item.id, item.label)
        s = mx.recordio.pack(header, b'')
#         print('s', s)
        q_out.put((i, s, oitem))

# main()

args = edict({
    'image_h': 112,
    'image_w': 112,
    'prefix': '../data/small_vgg',
    'root': '../data/small_vgg/small_vgg_112x112',
    'encoding':'.jpg',
    'quality': 95,
})

working_dir = args.prefix
prop = load_property(working_dir)
image_size = prop.image_size
print('image_size', image_size)

files = [os.path.join(working_dir, fname).replace('\\','/') for fname in os.listdir(working_dir)
        if os.path.isfile(os.path.join(working_dir, fname))]
print(files)

count = 0
for fname in files:
    # .lst file
    if fname.startswith(args.prefix) and fname.endswith('.lst'):
        print('Creating .rec file from', fname, 'in', working_dir)
        count += 1
        image_list = read_list(fname)

        # -- write_record -- #
        try:
            import Queue as queue
        except ImportError:
            import queue
        q_out = queue.Queue()
        fname = os.path.basename(fname)
        fname_rec = os.path.splitext(fname)[0] + '.rec'
        fname_idx = os.path.splitext(fname)[0] + '.idx'
        record = mx.recordio.MXIndexedRecordIO(os.path.join(working_dir, fname_idx),
                                               os.path.join(working_dir, fname_rec), 'w')
        cnt = 0
        pre_time = time.time()
        for i, item in enumerate(image_list):
            image_encode(args, i, item, q_out)
            if q_out.empty():
                continue
            _, s, item = q_out.get()
            #header, _ = mx.recordio.unpack(s)
            #print('write header label', header.label)
#             print(item[0])
            record.write_idx(item[0], s)
            if cnt % 1000 == 0:
                cur_time = time.time()
                print('time:', cur_time - pre_time, ' count:', cnt)
                pre_time = cur_time
            cnt += 1

image_size [112, 112]
['../data/small_vgg/property', '../data/small_vgg/train.idx', '../data/small_vgg/train.lst', '../data/small_vgg/train.rec']
Creating .rec file from ../data/small_vgg/train.lst in ../data/small_vgg
time: 0.0029981136322021484  count: 0
